In [1]:
from bs4 import BeautifulSoup

import re
import requests
import pickle

In [2]:
with open('../data/cats_with_277_pages', 'rb') as f:
     categories_and_max_pages = pickle.load(f)

In [3]:
model_link_and_title_pattern = re.compile(r'<a href="(.*)" title="(.*)" class=')

In [3]:
def parse_category_page(category_page):
    response = requests.get(category_page)
    model_urls_and_titles = re.findall('(?i)<a href=\"(https:\/\/www.cgtrader.com\/3d-models\/[a-z]+\/[^>]+)" title="([^"]+)', str(response.content))
    return model_urls_and_titles

In [4]:
def format_url_pagination(url, page):
    return f"{url}?page={page}"

In [12]:
test_urls = [format_url_pagination(x[0], 1) for x in categories_and_max_pages[0:10]]

In [13]:
test_urls

['https://www.cgtrader.com/3d-models/furniture?page=1',
 'https://www.cgtrader.com/3d-models/interior?page=1',
 'https://www.cgtrader.com/3d-models/architectural?page=1',
 'https://www.cgtrader.com/3d-models/character?page=1',
 'https://www.cgtrader.com/3d-models/house?page=1',
 'https://www.cgtrader.com/3d-models/exterior?page=1',
 'https://www.cgtrader.com/3d-models/modern?page=1',
 'https://www.cgtrader.com/3d-models/decoration?page=1',
 'https://www.cgtrader.com/3d-models/chair?page=1',
 'https://www.cgtrader.com/3d-models/car?page=1']

In [22]:
pages_to_scrape = []

for category, page in categories_and_max_pages:
    for i in range(1, page):
        formatted_url = format_url_pagination(category, i)
        pages_to_scrape.append(formatted_url)
        #model_title_tuples_lst.extend(parse_category_page(formatted_url))

In [24]:
from multiprocessing import Pool

model_title_tuples_lst = []

with Pool(4) as p:
    model_title_tuples_lst.append(p.map(parse_category_page, pages_to_scrape))

In [69]:
flattened_model_endpoints = []

for category in model_title_tuples_lst[0]:
    for model_tuple in category:
        if len(model_tuple[0]) > 1:
            flattened_model_endpoints.append(model_tuple[0])

In [70]:
len(flattened_model_endpoints)

354652

In [72]:
unique_flat_endpoint_urls = list(set(flattened_model_endpoints))

In [82]:
unique_flat_endpoint_urls[187610]

'https://www.cgtrader.com/3d-models/furniture/chair/0582-chair'

In [ ]:
unique_flat_endpoint_urls

In [84]:
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir('/media/void/Seagate Expansion Drive/data/cgtrader/') if isfile(join('/media/void/Seagate Expansion Drive/data/cgtrader/', f))]

In [85]:
onlyfiles

['luxurious-earrings-089',
 '00-c1e5db69-22fb-49a1-9cc4-7f569bef6489',
 '00008daan001',
 '0001-living-and-dinning-room',
 '0001-modern-chair',
 '0002-chair',
 '0003-glass-table',
 '00031daniel004-sitting',
 '00034joris003',
 '00035esmay004',
 '00046skander004-man-puts-on-jacket',
 '00049linda004',
 '00050pauljan010',
 '00056jeffrey007-business-man-pre-posed-3d-model',
 '0006-table-and-chairs-set',
 '00062camille008',
 '0007-le-corbusier-lounge-chair',
 '0008-le-corbusier-lc2-armchair',
 '0009-le-corbusier-lc2-armchai',
 '001-beat-machine',
 '001-house-a',
 '0010-modern-chair',
 '0012-modern-sofa',
 '0013-ottoman',
 '0015-modern-armchair',
 'industrial-pallet-truck-trolley-2',
 'industrial-park',
 'industrial-part-1',
 'industrial-pendant-light',
 'industrial-pendant-light-connell',
 'industrial-pendant-light-d956810d-4226-4588-8f04-5843992ba9b2',
 'industrial-pendant-light-fixture',
 'industrial-pipe',
 'industrial-pipe-assembly',
 'industrial-pipe-assembly-v2',
 'industrial-pipes-15-p

In [89]:
endpoints_to_retry = [x[0] for x in list(map(lambda x: (x, x.split('/')[-1]),unique_flat_endpoint_urls)) if x[1] not in onlyfiles]

In [93]:
def save_response_to_disk(endpoint_url):
    try:
        response = requests.get(endpoint_url, 'html')
        filename = endpoint_url.split('/')[-1]
        with open(f'/media/void/Seagate Expansion Drive/data/cgtrader2/{filename}', 'wb') as f:
            pickle.dump(response, f)
    except:
        print(f"could not retrieve {filename}")

In [95]:
len(endpoints_to_retry)

64292

In [96]:
with Pool(8) as p:
    p.map(save_response_to_disk, endpoints_to_retry)

UnboundLocalError: local variable 'filename' referenced before assignment

In [29]:
with open('../data/model_url_endpoints.pkl', 'wb') as f:
    pickle.dump(model_title_tuples_lst[0], f)

In [33]:
len(model_title_tuples_lst[0][0]) * len(model_title_tuples_lst[0])

511980

In [7]:
category, page = categories_and_max_pages[0]

In [10]:
formatted_endpoint_url = format_url_pagination(category, 1)